In [2]:
#import the libraries you'll need

import pandas as pd
import numpy as np
import pickle

## Let's walk through the other methods of handling missing data:


 
 - Fill with zeros/ones
 - Fill with True/False
 - Fill with mode/median
 - Fill with interpolated values
 - Fill forward/backward
 - Fill randomly
 

It's worth mentioning here, that not all of these options are reccomended or equally valid. Choosing your imputation methodology will affect the underlying distribution of the data, and can have profound impacts downstream. There are a huge number of things to consider when imputing data. This lab is about the tchnical HOW of data imputation. The when/why/where questions are much more complex and generally require some level of subject matter expertise on the underlying data, or at least some level of statistical understanding.

High level, there are a few things you wan't to think about when dealing with missing data:

 - Is the data missing at complete random?
 - Is there an underlying cause to the "missingness". In other words, does the underlying value drive its likeliness to be missing?
 - Is the missing data related to the other data points for that observation?
 
One of UBER's data scientists has done a nice write up of some of these considerations [here](https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4)

There is on oft quoted snippet from two Harvard statistics professors that sums up the dangers of imputation nicely:

“The idea of imputation is both seductive and dangerous. It is seductive because it can lull the user into the pleasurable state of believing that the data are complete after all, and it is dangerous because it lumps together situations where the problem is sufficiently minor that it can legitimately handled in this way and situations where standard estimators applied to real and imputed data have substantial bias.”

        ----Donald Rubin and Arthur Dempster (Incomplete Data in Sample Surveys: Theory and bibliographies)

In [5]:
#import the .pickle file we made in part 4

data = pd.read_pickle('credit_data_with_nans.pickle')
data.head()

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000.0,2.0,2.0,1.0,24.0,NaN,2.0,NaN,-1.0,-2.0,...,NaN,NaN,NaN,NaN,689.0,NaN,NaN,0.0,0.0,NaN
2,120000.0,2.0,2.0,NaN,26.0,-1.0,NaN,0.0,0.0,NaN,...,3272.0,3455.0,3261.0,0.0,NaN,1000.0,NaN,0.0,2000.0,1.0
3,90000.0,2.0,NaN,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,1518.0,1500.0,1000.0,1000.0,NaN,5000.0,0.0
4,50000.0,2.0,2.0,1.0,37.0,NaN,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,NaN,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
5,50000.0,1.0,2.0,1.0,NaN,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,NaN,36681.0,10000.0,9000.0,689.0,NaN,NaN


In [ ]:
#fill the NaNs with zeros in a new df

